In [ ]:
import os
import json
import pandas as pd
import pickle
import cv2


from tqdm import tqdm
from datetime import datetime

In [ ]:
PIPELINE_OUTPUT = '/Users/chanwutk/Documents/data/pipeline-outputs'
VIDEO = '/Users/chanwutk/Documents/data/videos'

In [ ]:
with open(os.path.join(PIPELINE_OUTPUT, 'segment-tracking-without-de-segment.json'), 'r') as f:
    trajectory_json = json.load(f)
trajectory_json[[*trajectory_json.keys()][0]][4]

In [ ]:
with open(os.path.join(PIPELINE_OUTPUT, 'segment-tracking-without-de-3d.json'), 'r') as f:
    trajectory_point_json = json.load(f)

In [ ]:
with open(os.path.join(VIDEO, 'frames.pkl'), 'rb') as f:
    frames = pickle.load(f)
frames['scene-0061-CAM_FRONT']

In [ ]:
def build_trajectories(metadata):
    id_to_trajectory = {}
    for m in metadata:
        for oid, det in m.items():
            if oid not in id_to_trajectory:
                id_to_trajectory[oid] = []
            id_to_trajectory[oid].append(det)
    
    for dets in id_to_trajectory.values():
        dets.sort(key=lambda d: d['timestamp'])
    return id_to_trajectory

In [ ]:
video_output_path = os.path.join(PIPELINE_OUTPUT, 'videos')

if not os.path.exists(video_output_path):
    os.makedirs(video_output_path)

for scene_name, trajs in trajectory_json.items():
    print(scene_name)
    id_to_trajectory = build_trajectories(trajs)
    _frames = frames[scene_name]
    video_filename = _frames['filename']
    cap = cv2.VideoCapture(os.path.join(VIDEO, video_filename))
    
    if (cap.isOpened()== False): 
        print("Error opening video stream or file")
    video_frames = []
    while(cap.isOpened()):
        # Capture frame-by-frame
        ret, frame = cap.read()
        if ret == True:
            video_frames.append(frame)
        else: 
            break
    
    assert len(video_frames) == len(_frames['frames'])

    # When everything done, release the video capture object
    cap.release()

    # Closes all the frames
    cv2.destroyAllWindows()
    
    video_scene_path = os.path.join(video_output_path, scene_name)
    if not os.path.exists(video_scene_path):
        os.makedirs(video_scene_path)
    
    for oid, trajectory in id_to_trajectory.items():
        min_time = min(t['timestamp'] for t in trajectory)
        max_time = max(t['timestamp'] for t in trajectory)
        print(oid, min_time, max_time)
        filtered_frames = [(i, f) for i, f in enumerate(_frames['frames']) if min_time <= str(f[9]) <= max_time]
#         print(trajectory[0])
#         print([str(i) + ' ' + str(f[9]) for i, f in filtered_frames])
    
        out = cv2.VideoWriter(os.path.join(video_scene_path, f'{oid}.mp4'),cv2.VideoWriter_fourcc(*'mp4v'), 12, (1600, 900))
        for i, f in tqdm(filtered_frames):
            video_frame = video_frames[i].copy()
            segments = [s for s in trajectory if s['detection_id'][0] == i]
            assert len(segments) <= 1, segments
            if len(segments) == 0:
                assert oid not in trajectory_point_json[scene_name][i]
            else:
                segment = segments[0]
                det = trajectory_point_json[scene_name][i][oid]
                assert det['detection_id'] == segment['detection_id']
                assert det['detection_id'][0] == i
                
                start_point = tuple(map(int, (det['bbox_left'], det['bbox_top'])))
                end_point = tuple(map(int, (det['bbox_left'] + det['bbox_w'], det['bbox_top'] + det['bbox_h'])))
#                 print(start_point, end_point)
                color = (255, 0, 0)
                thickness = 2
                video_frame = cv2.rectangle(video_frame, start_point, end_point, color, thickness)
            out.write(video_frame)
        out.release()
